In [ ]:
import keras.backend as K
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import src

In [ ]:
#model folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/models/')
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D, Conv2D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization

The baseline model predicts x next frames from x past frames.
Source: https://github.com/wqxu/ConvLSTM/blob/master/radar_forecast.py

In [ ]:
past = 2
name = f"baseline_{past}-1"

In [ ]:
n_pixel = 64

seq = Sequential()
seq.add(ConvLSTM2D(filters=n_pixel, kernel_size=(3, 3),
                   input_shape=(None, n_pixel, n_pixel, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=n_pixel, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=n_pixel, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=n_pixel, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))

Loads pretrained model weights.

In [ ]:
seq.load_weights(sys.path[1]+name+"/"+name+"_model.h5")

Compile model.

In [ ]:
seq.compile(loss='mean_squared_error', optimizer='adadelta')

In [ ]:
seq.summary()

Go to the prediction part if you only want to use the loaded model for prediction.

## Load dataset

In [ ]:
train, xval, test = src.load_datasets(past_frames=past,future_frames=past)

Split data to input and ground truth and augment optionally.

In [ ]:
baseline_train, baseline_truth, baseline_val, baseline_val_truth, baseline_test, baseline_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, future_frames=past, augment=True)

The ConvLSTM receives 5D samples.

In [ ]:
baseline_train = np.reshape(np.transpose(baseline_train,(0,3,1,2)), np.transpose(baseline_train,(0,3,1,2)).shape+(1,))
baseline_truth = np.reshape(np.transpose(baseline_truth,(0,3,1,2)), np.transpose(baseline_truth,(0,3,1,2)).shape+(1,))
baseline_val = np.reshape(np.transpose(baseline_val,(0,3,1,2)), np.transpose(baseline_val,(0,3,1,2)).shape+(1,))
baseline_val_truth = np.reshape(np.transpose(baseline_val_truth,(0,3,1,2)), np.transpose(baseline_val_truth,(0,3,1,2)).shape+(1,))
baseline_test = np.reshape(np.transpose(baseline_test,(0,3,1,2)), np.transpose(baseline_test,(0,3,1,2)).shape+(1,))
baseline_test_truth = np.reshape(np.transpose(baseline_test_truth,(0,3,1,2)), np.transpose(baseline_test_truth,(0,3,1,2)).shape+(1,))

## Training

We train the baseline model for 20 epochs with a batch size of 16. This way there are $8000/16*20=10000$ gradient updates in total. Same as in our best model trainings where we did 5000 iterations and trained the generator twice per iteration.

In [ ]:
seq.fit(baseline_train, baseline_truth, batch_size=16, epochs=20, validation_data=(baseline_val, baseline_val_truth))

## Prediction

Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)
#sequence_test = src.augment_data(sequence_test[:100])

Reshape data to 5D.

In [ ]:
sequence_test = np.reshape(np.transpose(sequence_test,(0,3,1,2)), np.transpose(sequence_test,(0,3,1,2)).shape+(1,))

In [ ]:
n_next = 5
predictions = {}
past_frames = sequence_test[:,0:past]
test_truth = sequence_test[:,past:past+past]

To be consistent with our own model trainings we always use the first image of the prediction (model predicts 2 images) and make only one step forward in each iteration.

In [ ]:
for t in range(n_next):
    future = seq.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future[:,:1,...]
    print(predictions[f"{t}"].shape)
    past_frames = np.concatenate((past_frames[:,1:,...], predictions[f"{t}"]), axis=1)
    print(past_frames.shape)

Reshape predictions to 4D.

In [ ]:
for key,value in enumerate(predictions):
    predictions[str(key)] = np.reshape(predictions[str(key)],((predictions[str(key)].shape[0],)+predictions[str(key)].shape[2:4]+(predictions[str(key)].shape[1],)))

Save example predictions. (First reload test data or resahpe it back to 4D)

In [ ]:
sequence_test = src.load_datasets(prediction=True)
#sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
# 44, 110 in whole data w.o. augmenting, 33, 67, 57 in augmented of the first 100
src.sequence_prediction_plot(name, sequence_test, predictions, past, samples=[33,67,57])

Renormalize intensity values


In [ ]:
test_norms = np.load(sys.path[0]+"/5min_long_pred_norms_compressed.npz")["arr_1"]

In [ ]:
#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in list(map(str,np.arange(0,n_next)))])[:,:,:,:,0]), (1,2,3,0))

Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>

In [ ]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

In [ ]:
intensity_scores = src.get_scores(renormalized_predictions, renormalized_test, n_next, past, thresholds_as_list=[18])
scores = src.get_scores(I_p, I_t, n_next, past, thresholds_as_list=[0.5])

Save scores.

In [ ]:
np.save(name+"_scores",scores)
np.save(name+"_intensity_scores",intensity_scores)